In [1]:
from transformers import AutoModelForSeq2SeqLM
from peft import get_peft_config, get_peft_model, get_peft_model_state_dict, LoraConfig, TaskType
import torch
from datasets import load_dataset
import os

os.environ["TOKENIZERS_PARALLELISM"] = "false"
from transformers import AutoTokenizer
from torch.utils.data import DataLoader
from transformers import default_data_collator, get_linear_schedule_with_warmup
from tqdm import tqdm
from datasets import load_dataset

device = "cpu"
model_name_or_path = "bigscience/mt0-large"
tokenizer_name_or_path = "bigscience/mt0-large"

checkpoint_name = "financial_sentiment_analysis_lora_v1.pt"
text_column = "sentence"
label_column = "text_label"
max_length = 128
lr = 1e-3
num_epochs = 3
batch_size = 8

In [2]:
import torch
torch.cuda.empty_cache()

In [3]:
# creating model
peft_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()
model

trainable params: 2359296 || all params: 1231940608 || trainable%: 0.19151053100118282


PeftModelForSeq2SeqLM(
  (base_model): LoraModel(
    (model): MT5ForConditionalGeneration(
      (shared): Embedding(250112, 1024)
      (encoder): MT5Stack(
        (embed_tokens): Embedding(250112, 1024)
        (block): ModuleList(
          (0): MT5Block(
            (layer): ModuleList(
              (0): MT5LayerSelfAttention(
                (SelfAttention): MT5Attention(
                  (q): Linear(
                    in_features=1024, out_features=1024, bias=False
                    (lora_dropout): ModuleDict(
                      (default): Dropout(p=0.1, inplace=False)
                    )
                    (lora_A): ModuleDict(
                      (default): Linear(in_features=1024, out_features=8, bias=False)
                    )
                    (lora_B): ModuleDict(
                      (default): Linear(in_features=8, out_features=1024, bias=False)
                    )
                    (lora_embedding_A): ParameterDict()
                    (lora_emb

In [4]:
# loading dataset
dataset = load_dataset("financial_phrasebank", "sentences_allagree")
dataset = dataset["train"].train_test_split(test_size=0.1)
dataset["validation"] = dataset["test"]
del dataset["test"]

classes = dataset["train"].features["label"].names
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["label"]]},
    batched=True,
    num_proc=1,
)

dataset["train"][0]

Found cached dataset financial_phrasebank (C:/Users/MSI/.cache/huggingface/datasets/financial_phrasebank/sentences_allagree/1.0.0/550bde12e6c30e2674da973a55f57edde5181d53f5a5a34c1531c53f93b7e141)


  0%|          | 0/1 [00:00<?, ?it/s]

Map:   0%|          | 0/2037 [00:00<?, ? examples/s]

Map:   0%|          | 0/227 [00:00<?, ? examples/s]

{'sentence': "It 's not .", 'label': 1, 'text_label': 'neutral'}

In [5]:
# data preprocessing
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)


def preprocess_function(examples):
    inputs = examples[text_column]
    targets = examples[label_column]
    model_inputs = tokenizer(inputs, max_length=max_length, padding="max_length", truncation=True, return_tensors="pt")
    labels = tokenizer(targets, max_length=3, padding="max_length", truncation=True, return_tensors="pt")
    labels = labels["input_ids"]
    labels[labels == tokenizer.pad_token_id] = -100
    model_inputs["labels"] = labels
    return model_inputs


processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["validation"]

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

Running tokenizer on dataset:   0%|          | 0/2037 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/227 [00:00<?, ? examples/s]

In [6]:
# optimizer and lr scheduler
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [7]:
import torch
torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |

In [8]:
# training and evaluation
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 29/29 [02:23<00:00,  4.95s/it]


epoch=0: train_ppl=tensor(1.3619) train_epoch_loss=tensor(0.3089) eval_ppl=tensor(1.0607) eval_epoch_loss=tensor(0.0589)


100%|██████████| 29/29 [02:24<00:00,  4.99s/it]


epoch=1: train_ppl=tensor(1.0664) train_epoch_loss=tensor(0.0643) eval_ppl=tensor(1.0224) eval_epoch_loss=tensor(0.0221)


100%|██████████| 29/29 [02:44<00:00,  5.66s/it]

epoch=2: train_ppl=tensor(1.0356) train_epoch_loss=tensor(0.0350) eval_ppl=tensor(1.0299) eval_epoch_loss=tensor(0.0295)


In [9]:
# print accuracy
correct = 0
total = 0
for pred, true in zip(eval_preds, dataset["validation"]["text_label"]):
    if pred.strip() == true.strip():
        correct += 1
    total += 1
accuracy = correct / total * 100
print(f"{accuracy=} % on the evaluation dataset")
print(f"{eval_preds[:10]=}")
print(f"{dataset['validation']['text_label'][:10]=}")

accuracy=97.3568281938326 % on the evaluation dataset
eval_preds[:10]=['neutral', 'positive', 'positive', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'neutral']
dataset['validation']['text_label'][:10]=['neutral', 'positive', 'positive', 'neutral', 'neutral', 'positive', 'neutral', 'neutral', 'positive', 'neutral']


In [10]:
# saving model
peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"
model.save_pretrained(peft_model_id)

In [11]:
ckpt = f"{peft_model_id}/adapter_model.bin"
!du -h $ckpt

'du' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [12]:
from peft import PeftModel, PeftConfig

peft_model_id = f"{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

In [19]:
model.eval()
inputs = tokenizer("""Insurance motto: "Who is trying nothing, has nothing!"So they prefer to be punished by the courts and pay with compensation, rather than doing what they are commissioned for: means and results on a lasting recovery of disorders! It is actually unhappy, but from the principle that we know their operation….We act accordingly for our customers 💪bravo to you ... See more	""", return_tensors="pt")
print(inputs)

with torch.no_grad():
    outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    print(outputs)
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

In 2008 , AVC Systemhaus had net sales of EUR 10 million USD 7.1 m .
{'input_ids': tensor([[ 46968,   2921,    476,    267,    313, 126629,    339,   6557,    347,
            259,  24146,    261,   1070,    259,  24146,   9496,  14721,    287,
            276,  14310,    288,    390,   3102,  71572,    455,    287,  15557,
            263,    305,   5112,    514,    259, 114345,    261,   1101,  19737,
           2421,    342,    347,   2119,    287,    276,    418,  45613,    345,
            332,    267,    259,  12909,    305,  10678,    351,    259,    262,
            259,  70529,    259,  60903,    304,    259, 152832,    309,   1385,
            339,    259,  15128,    335,  65685,    261,   1156,    702,    287,
            259,  45966,    533,    787,   3076,    259,   1616,  25236,   1949,
           9248,  16097,    259,  18775,    484,    332,   1406,  19934,    259,
         174589, 115725,    288,    521,    480,   5483,   1097,    259,      1]]), 'attention_mask': tenso

In [27]:
def classify(x):
    inputs = tokenizer(x, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(input_ids=inputs["input_ids"], max_new_tokens=10)
    return tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0].title()

In [29]:
import time
from tqdm import tqdm
import pandas as pd
import os
data_dir = r'C:\Users\MSI\Desktop\ereputation\ced_group'
df = pd.read_excel(os.path.join(data_dir, "processed.xlsx"))
l = []
start = time.time()
for i in tqdm(range(len(df))):
    l.append(classify(df.text.iloc[i]))
df["sentiment"] = l
end = time.time()
print(f'Sentiment analysis task took {int(end - start)} seconds.\n')
df

100%|██████████| 1162/1162 [14:57<00:00,  1.29it/s]


Sentiment analysis task took 897 seconds.



,original_text,is_post,keyword,author,period,source,lang,text,text_id,preprocess_sa,preprocess_tm,processed,sentiment
0,"During the webinar, visitors can learn from th...",1,ced europe,Insight,03-2023,1,en,"During the webinar, visitors can learn from th...",5afed6ae90800e260ded,webinar visitor learn cybersecurity approach r...,webinar visitor learn cybersecurity approach r...,webinar visitor cybersecurity approach company...,Neutral
1,"South Belgian Rally terminé ✅ P7️⃣, pas du tou...",1,ced europe,Cedric De Cecco,03-2022,1,fr,"South Belgian Rally finished ✅ P7️⃣, not at al...",65a199c0c2efad94f670,south belgian rally finished ✅ p7️⃣ expected r...,south belgian rally finished expected result m...,south rally result must loop finish rally smil...,Positive
2,"Jump into this new season 🚀Tomorrow, we will s...",1,ced europe,Cedric De Cecco,03-2022,1,en,"Jump into this new season 🚀Tomorrow, we will s...",42f4fd970c3d51a738c6,jump new season 🚀tomorrow start first rally 20...,jump new season tomorrow start first rally tun...,season rally belux ced europe,Neutral
3,Drive fast and keep safe! Good luck! Enjoy it !,0,ced europe,Cedric De Cecco,03-2022,1,en,Drive fast and keep safe! Good luck! Enjoy it !,687c7656f264e702da92,drive fast keep safe good luck enjoy,drive fast keep safe good luck enjoy,luck,Positive
4,"When a house or business premises are damaged,...",1,ced europe,CED Europe,03-2022,1,en,"When a house or business premises are damaged,...",36a46d4735c7240d2db9,house business premise damaged help often need...,house business premise damaged help often need...,house business premise help whether lock leaka...,Neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1157,#Mirage has arrived! Very excited to begin fin...,1,cedgroup,Cambridge University Eco Racing,09-2017,2,en,#Mirage has arrived! Very excited to begin fin...,84a79c5d8ac706bdb08b,arrived excited begin final assembly big thank...,arrived excited begin final assembly big thank...,assembly thank cedgroup tollgroup shipment,Positive
1158,"Fotos del “Work Shop CEDGroup”, Mayo 2014. Gra...",1,cedgroup,Tree-Oss,06-2014,2,es,"Photos of the “Work Shop Cedgroup”, May 2014. ...",893ad4419acf9ae4a3dd,photo “ work shop cedgroup ” may 2014 thank pa...,photo work shop cedgroup may thank participating,photo shop may,Positive
1159,@jeffjarvis wd love to talk about your CUNY a...,1,cedgroup,rhesa j,08-2010,2,en,@jeffjarvis wd love to talk about your CUNY a...,9b6c1adb489f1c733002,wd love talk cuny approach see wwwcedgroupcom ...,wd love talk cuny approach see wwwcedgroupcom ...,jeffjarvis love talk cuny approach wwwcedgroup...,Neutral
1160,Our #SponsorMonday blog this week is all about...,1,cedgroup,Cambridge University Eco Racing,11-2015,2,en,Our #SponsorMonday blog this week is all about...,97cbabf02bf5c8234487,blog week helped get safely,blog week helped get safely,blog week cedgroup,Neutral


In [30]:
df.sentiment.value_counts()

Positive    638
Neutral     499
Negative     25
Name: sentiment, dtype: int64